In [44]:
# from google.colab import drive
# drive.mount('/content/drive')

In [45]:
# cd drive/MyDrive/HW/661Project/

In [46]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import time

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.conv3 = nn.Conv2d(16, 32, 3)
        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.bn2(self.conv2(out)))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.bn3(self.conv3(out)))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        # out = self.fc3(out)
        out = self.fc4(out)
        return out

In [48]:
##Do Not Touch This Cell

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.5)
if device =='cuda':
    print("Train on GPU...")
else:
    print("Train on CPU...")

Train on GPU...


In [49]:
##Do Not Touch This Cell
max_epochs = 100

random_seed = 671
torch.manual_seed(random_seed)

In [50]:
crop_size = 64

data_transforms ={
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=crop_size, scale=(0.5, 1.0)),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5],[.5, .5, .5])
    ]),
    'test': transforms.Compose([
        transforms.RandomResizedCrop(size=crop_size, scale=(0.5, 1.0)),
        # transforms.Resize(256),
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([.5, .5, .5],[.5, .5, .5])
    ])
}

train_set = torchvision.datasets.ImageFolder(root='./training set reenact/', transform=data_transforms['train'])
train_loader = DataLoader(dataset=train_set, batch_size=128, shuffle=True, num_workers=2)

test_set = torchvision.datasets.ImageFolder(root='./testing set reenact/', transform=data_transforms['test'])
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=True, num_workers=2)

In [51]:
loss_list, acc_list = [], []
loss_list_val, acc_list_val = [], []
criterion = nn.CrossEntropyLoss()

start = time.time()

for epoch in range(max_epochs):
    #TODO: set the net to train mode:
    net.train()

    epoch_loss = 0.0
    correct = 0
    total_examples = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)

        optimizer.zero_grad()
        ##TODO: pass the data into the network and store the output
        outputs = net(data)
        ##TODO: Calculate the cross entropy loss between the output and target 
        loss = criterion(outputs, labels)
        ##TODO: Perform backpropagation
        loss.backward()
        optimizer.step()

        ##TODO: Get the prediction from the output
        _, predicted = torch.max(outputs, 1)


        # print(f'outputs:{outputs}')
        # print(f'predicted:{predicted}')
        # print(f'labels:{labels}')


        ##TODO: Calculate the correct number and add the number to correct
        correct += predicted.eq(labels).sum()
        total_examples += labels.size(0)
        ##TODO: Add the loss to epoch_loss.
        epoch_loss += loss.item()
    ##TODO: calculate the average loss
    avg_loss = epoch_loss / len(train_loader)
    ##TODO: calculate the average accuracy
    avg_acc = correct / total_examples
    ##TODO: append average epoch loss to loss list
    loss_list.append(avg_loss)
    ##TODO: append average accuracy to accuracy list
    acc_list.append(avg_acc)
        
    print('[epoch %d] loss: %.5f accuracy: %.4f time: %.4fs' % (epoch + 1, avg_loss, avg_acc, time.time()-start))

[epoch 1] loss: 0.69386 accuracy: 0.4959 time: 1.4653s
[epoch 2] loss: 0.69069 accuracy: 0.4959 time: 2.9467s
[epoch 3] loss: 0.69192 accuracy: 0.5124 time: 4.4187s
[epoch 4] loss: 0.69262 accuracy: 0.4959 time: 6.0732s
[epoch 5] loss: 0.68925 accuracy: 0.5041 time: 7.5776s
[epoch 6] loss: 0.68892 accuracy: 0.5124 time: 9.2398s
[epoch 7] loss: 0.68888 accuracy: 0.5041 time: 10.7014s
[epoch 8] loss: 0.68884 accuracy: 0.5372 time: 12.2484s
[epoch 9] loss: 0.68660 accuracy: 0.5868 time: 13.7420s
[epoch 10] loss: 0.68735 accuracy: 0.5537 time: 15.5034s
[epoch 11] loss: 0.68706 accuracy: 0.5702 time: 16.9946s
[epoch 12] loss: 0.68639 accuracy: 0.6116 time: 18.4767s
[epoch 13] loss: 0.68711 accuracy: 0.5950 time: 20.1233s
[epoch 14] loss: 0.68599 accuracy: 0.6281 time: 21.6118s
[epoch 15] loss: 0.68318 accuracy: 0.6694 time: 23.1615s
[epoch 16] loss: 0.68422 accuracy: 0.6033 time: 24.7669s
[epoch 17] loss: 0.68484 accuracy: 0.6364 time: 26.2347s
[epoch 18] loss: 0.68243 accuracy: 0.6364 time

In [52]:
correct_test = 0
net.eval()
with torch.no_grad():
    for batch_idx, (data, label) in enumerate(test_loader):
        data, label = data.to(device), label.to(device)
        ##TODO: pass the data into the network and store the output
        outputs = net(data)
        ##TODO: Get the prediction from the output
        _, predicted = torch.max(outputs, 1)
        ##TODO: Calculate the correct number and add the number to correct_test
        correct_test += predicted.eq(label).sum()

print('Accuracy on the test images: %.2f %%' % (100 * correct_test / len(test_set)))

Accuracy on the test images: 75.00 %
